# Create JSON-LD for a Metadata Collection with the schema.org Vocabulary Google Recommends for Datasets using the Conceptual Evaluation for the Recommendation
### Notebook Goals
* Create csv with recommendation concept content for the collection 
* Translate concept names into the schema.org vocabulary
* Create valid JSON-LD for a record
* Use Google's Structured Data Testing Tool to test results

In [ ]:
# refine dataframe, create record json
import pandas as pd
# create dataframe structure that contains the records content
import MDeval as md

#### Describe the metadata. 
* What organization created the records? (Organization)
* What collection are the records from? (Collection)
* What dialect are the records written in? (Dialect)

In [ ]:
# variables for function arguments, fill these out
Organization = 'LTER'
Collection = 'MILES'
Dialect = 'EML'

#### Read in the metadata's recommendation evaluated csv

In [ ]:
''' 
Read in the recommendation evaluated csv created in the last Notebook
defined by the variables from the previous cell
''' 
RecommendationEvaluatedDF = pd.read_csv(
    './data/'+Organization+'/'+Collection+'_'+Dialect+'_RecommendationEvaluated.csv'
)

#### Record Concept Content Function
* Rows are records
* Columns are concepts
* Values are content from the record, comma separated if there are multiple instances

In [ ]:
'''
MDeval.recordConceptContent requires a dataframe with concepts. It creates a vertical view of
recommendation concept content for each record in the collection.
'''
recordDF = md.recordConceptContent(RecommendationEvaluatedDF)
# organize the table (readability)
RecRecordDF = recordDF[['Collection', 'Record', 'Resource Title', 'Abstract',
                        'Online Resource', 'Keyword',
                        'Author / Originator', 'Distribution Format',
                        'Resource Type', 'Resource Version',
                        'Temporal Extent', 'Spatial Extent',
                        'Resource Citation']]
# display the dataframe
RecRecordDF

#### Choose a record to translate

In [ ]:
# set RecordChoice variable
RecordChoice = '1.xml'

In [ ]:
# select record row using the RecordChoice variable, drop other rows
RecRecordDF = RecRecordDF[RecRecordDF['Record'] == RecordChoice]
# drop the Collection and Record columns
RecRecordDF = RecRecordDF.drop(['Collection', 'Record'], 'columns')
# display the chosen record's content
RecRecordDF

#### Translate concepts to schema.org vocabulary

In [ ]:
# apply a dictionary with column names that you want to rename with schema.org vocabulary words
RecRecordDF = RecRecordDF.rename(
    {'Resource Title':'name', 'Abstract':'description', 'Online Resource':'url',
     'Keyword':'keywords', 'Author / Originator':'creator', 'Distribution Format': 'distribution',
     'Resource Type':'@type', 'Resource Version': 'version', 'Temporal Extent': 'temporalCoverage',
     'Spatial Extent': 'spatialCoverage', 'Resource Citation':'citation'}, axis='columns'
    )
# display the dataframe
RecRecordDF

#### Add the required context

In [ ]:
'''
insert a column in the third position with "@context" as the header
and "http://schema.org as the content
'''
RecRecordDF.insert(2, '@context', 'http://schema.org/')
# display the dataframe
RecRecordDF

#### Create JSON-LD String

In [ ]:
# create JSON using the pandas function "to_json" use the "record" orientation
recordDict = RecRecordDF.to_json(orient='records')
# add the HTML script element that declares the contents to be JSON-LD
RecordJSONld = '<script type="application/ld+json">' + recordDict[1:-1] + '</script>'
# display the resultant string
RecordJSONld

#### Test JSON-LD for validity
* Take string produced by the cell above and copy it.
* Go to [Google's Structured Data Testing Tool](https://search.google.com/structured-data/testing-tool#new-test)
* Select the "Code Snippet Tab"
* Paste string and "Run Test"
* Click on errors to highlight the portion of the string that needs improvement
* Rerun test with the play button in the middle bottom of the screen

[Bonus Notebook: Create a dialect specific translation using the recordXpathContent MDeval function](./03.CreateXpathJSON-LD.ipynb)